In [ ]:
from rkllama_core import LLM

# 기본 사용

In [ ]:
# LLM 인스턴스 생성
llm = LLM(model="Qwen2.5-3B-Instruct_W8A8_G128_RK3588")

# 간단한 질문
response = llm.invoke("지구의 자전 주기는?")
print(f"🤖 응답: {response}")

# 직접 호출 (callable)
response2 = llm("파이썬의 장점 3가지만 말해줘")
print(f"🤖 응답: {response2}")

# 시스템 프롬프트 사용

In [ ]:
system_prompt = "당신은 친근한 AI 어시스턴트입니다. 항상 존댓말을 사용하세요."

llm = LLM(
        model="Qwen2.5-3B-Instruct_W8A8_G128_RK3588",
        system_prompt=system_prompt
    )
    
response = llm.invoke("안녕하세요!")
print(f"🤖 응답: {response}")

# 창의성 temperature 사용

In [ ]:
question = "창의적인 이야기를 짧게 써주세요"

In [ ]:
# 낮은 온도 (보수적)
llm_conservative = LLM(temperature=0.1)
response1 = llm_conservative.invoke(question)
print(f"🌡️ 온도 0.1: {response1}")

In [ ]:
# 높은 온도 (창의적)
llm_creative = LLM(temperature=1.5)
response2 = llm_creative.invoke(question)
print(f"🌡️ 온도 1.5: {response2}")

# LLM Chaining

In [ ]:
llm = LLM()

In [ ]:
    
# bind 메서드 사용
korean_llm = llm.bind(
    system_prompt="모든 답변을 한국어로 해주세요",
    temperature=0.5
)

response = korean_llm.invoke("What is artificial intelligence?")
print(f"🤖 한국어 응답: {response}")


In [ ]:
# with_config 메서드 사용
formal_llm = llm.with_config(
    system_prompt="정중하고 격식있는 어조로 답변해주세요",
    temperature=0.3
)

response2 = formal_llm.invoke("AI의 미래에 대해 어떻게 생각하세요?")
print(f"🤖 격식있는 응답: {response2}")

# 스트리밍 사용

In [ ]:
llm = LLM()

In [ ]:
print("🤖 스트리밍 응답: ", end="", flush=True)
for chunk in llm.stream_invoke("AI와 로봇의 차이점을 설명해주세요"):
    print(chunk, end="", flush=True)
print()  # 줄바꿈

# Function Calling 사용

In [ ]:
from rkllama_core import LLM, tool, RKLlamaAgent

In [ ]:
# 도구 사용하는 예시
@tool
def get_weather(city: str) -> str:
    """날씨 정보를 가져옵니다."""
    import random
    temps = random.randint(-10, 35)
    conditions = random.choice(["맑음", "흐림", "비", "눈"])
    return f"{city}의 날씨: {temps}°C, {conditions}"

In [ ]:
# 1. 순수 LLM 사용
llm = LLM(system_prompt="당신은 도움이 되는 AI 어시스턴트입니다.")
response = llm.invoke("서울 날씨가 어때?")
print(f"🤖 LLM만 사용: {response}")

In [ ]:
# 2. Agent와 도구 사용
agent = RKLlamaAgent()
agent.bind_tools([get_weather])
response = agent.invoke("서울 날씨가 어때?")
print(f"🤖 Agent + Tools: {response}")

# 대화 저장 

In [ ]:
llm = LLM(
        system_prompt="당신은 친근한 대화 상대입니다. 이전 대화 내용을 기억하며 자연스럽게 대화해주세요.",
        temperature=0.7
    )

In [ ]:
conversation_history = []

In [ ]:
def chat(message: str) -> str:
    # 대화 기록을 시스템 프롬프트에 포함
    context = "\n".join([f"사용자: {h['user']}\nAI: {h['ai']}" for h in conversation_history[-3:]])  # 최근 3개만
    full_prompt = f"이전 대화:\n{context}\n\n현재 질문: {message}" if context else message
    
    response = llm.invoke(full_prompt)
    
    # 대화 기록 저장
    conversation_history.append({"user": message, "ai": response})
    
    return response

In [ ]:
# 대화 시뮬레이션
print(f"🧑 사용자: 안녕하세요!")
response1 = chat("안녕하세요!")
print(f"🤖 AI: {response1}")

In [ ]:
print(f"\n🧑 사용자: 오늘 날씨가 좋네요")
response2 = chat("오늘 날씨가 좋네요")
print(f"🤖 AI: {response2}")